<a href="https://colab.research.google.com/github/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/ECC-GNN Spectral example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/BIONETdatasets/main?filepath=TUD%2Fnotebooks%2FECC-GNN%20Spectral%20example.ipynb)
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/ECC-GNN Spectral example.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# Cloning BIONETdatasets repository

In [16]:
!git clone http://github.com/giordamaug/BIONETdatasets

Cloning into 'BIONETdatasets'...
remote: Enumerating objects: 441, done.
remote: Counting objects: 100% (441/441), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 441 (delta 170), reused 244 (delta 93), pack-reused 0
Receiving objects: 100% (441/441), 48.21 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (170/170), done.
Checking out files: 100% (43/43), done.


In [17]:
!pip install -q spektral
!pip install -q sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


# Load the dataset

In [2]:
#@title select the dataset { form-width: "30%" }
dataname = "MUTAG" #@param ["ogbg-molbace", "ogbg-molbbbp", "KIDNEY", "MUTAG", "PROTEINS", "Mutagenicity" ]
import shutil
import os
#shutil.unpack_archive(f'BIONETdatasets/TUD/datasets/{dataname}.zip')
shutil.unpack_archive(f'../datasets/{dataname}.zip')
import sys
sys.path.append('BIONETdatasets/TUD')
sys.path.append('..')
from wrappers.spektral_wrapper import MyTUDataset
dataset = MyTUDataset(dataname, path=dataname, verbose=True)

Successfully loaded MUTAG.


# Apply ECC-GNN model on dataset

In [3]:
"""
This example shows how to perform molecule classification with the
[Open Graph Benchmark](https://ogb.stanford.edu) `mol-hiv` dataset, using a
simple ECC-based GNN in disjoint mode. The model does not perform really well
but should give you a starting point if you want to implement a more
sophisticated one.
"""

import numpy as np
from time import time
import tqdm as tq
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy, binary_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.layers import ECCConv, GlobalSumPool

from sklearn.metrics import confusion_matrix,matthews_corrcoef,accuracy_score

################################################################################
# Config
################################################################################
#@title Parameters { form-width: "30%" }
learning_rate = 0.001  #@param {type:"number"}
epochs = 50 #@param {type:"slider", min:0, max:500, step:20}
channels = 32 #@param {type:"slider", min:16, max:128, step:16}
batch_size = 1  #@param {type:"slider", min:1, max:64, step:1}
folds = 2  #@param {type:"slider", min:1, max:10, step:1}
verbose = True #@param {type:"boolean"}
seed = 42 #@param {type:"number"}

# Parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target
tf.keras.backend.set_floatx('float64')
################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self, channels=32, activation="sigmoid"):
        super().__init__()
        self.conv1 = ECCConv(channels, activation="relu")
        self.conv2 = ECCConv(channels, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out, activation=activation)

    def call(self, inputs):
        x, a, e, output = inputs
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        output = self.global_pool(x)
        output = self.dense(output)

        return output

start = time()

################################################################################
# Cross Validation loop
################################################################################
from sklearn.model_selection import StratifiedKFold
targets = [g.y.dot(1 << np.arange(g.y.size)[::-1]) for g in dataset]
sp = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
results = []
for idx_tr, idx_te in tq.tqdm(list(sp.split(dataset, targets)), desc="fold: "):
    dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]
    loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs)
    loader_te = DisjointLoader(dataset_te, batch_size=batch_size, epochs=1)

    if n_out>2:
        model = Net(channels=channels, activation='softmax')
        loss_fn = CategoricalCrossentropy()
        accuracy_metric = categorical_accuracy
    else:
        model = Net(channels=channels, activation='sigmoid')
        loss_fn = BinaryCrossentropy()
        accuracy_metric = binary_accuracy
    optimizer = Adam(learning_rate)
    ################################################################################
    # Fit model
    ################################################################################
    @tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
    def train_step(inputs, target):
        with tf.GradientTape() as tape:
            predictions = model(inputs, training=True)
            loss = loss_fn(target, predictions) + sum(model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        acc = tf.reduce_mean(accuracy_metric(target, predictions))
        return loss, acc

    epoch = step = 0
    tresults = []
    for batch in loader_tr:
        step += 1
        loss, acc = train_step(*batch)
        tresults.append((loss, acc))
        if step == loader_tr.steps_per_epoch:
            step = 0
            epoch += 1
            if verbose: print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(tresults, 0)))
            tresults = []

    ################################################################################
    # Evaluate model
    ################################################################################
    for batch in loader_te:
        inputs, target = batch
        predictions = model(inputs, training=False)
        results.append(
            (
                loss_fn(target, predictions),
                tf.reduce_mean(accuracy_metric(target, predictions)),
            )
        )
    if verbose: print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))
# Timing
temp = time() - start
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes
expired = '%d:%d:%d' %(hours,minutes,seconds)
print("Done. Test loss: {}. Test acc: {}".format(*np.mean(results, 0)))


fold:   0%|          | 0/2 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



fold:   0%|          | 0/2 [00:01<?, ?it/s]


TypeError: in user code:

    <ipython-input-3-24991538bf23>:94 train_step  *
        acc = tf.reduce_mean(accuracy_metric(target, predictions))
    /Users/maurizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /Users/maurizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/metrics.py:3239 binary_accuracy
        return K.mean(math_ops.equal(y_true, y_pred), axis=-1)
    /Users/maurizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/maurizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/math_ops.py:1613 equal
        return gen_math_ops.equal(x, y, name=name)
    /Users/maurizio/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/gen_math_ops.py:3222 equal
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /Users/maurizio/opt/anaconda3/lib/pyt